<a href="https://colab.research.google.com/github/Sen-Ma/Dacon/blob/master/Dacon_%EC%A0%9C%EC%A3%BC%EB%B2%84%EC%8A%A4%EC%9A%B4%ED%96%89%EC%8B%9C%EA%B0%84%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np 
import torch.backends.cudnn
import torch
import os
from glob import glob
from torch import nn, optim
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Dataset, TensorDataset
import pandas as pd 
import geopy.distance

In [0]:
train = pd.read_csv('/content/gdrive/My Drive/train.csv')
test = pd.read_csv('/content/gdrive/My Drive/test.csv')

In [0]:
np.random.seed(1)
torch.manual_seed(1)
torch.backends.cudnn.deterministic = True 
torch.backends.cudnn.benchmark = False

In [23]:
train.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897,64


In [24]:
indexNames = train[train['now_arrive_time'] == "00시"].index
train.drop(indexNames, inplace=True)
indexNmaes = test[test['now_station'] == "제주한라대학교(종점)"].index
test.set_value(indexNmaes, 'now_station', "노형오거리")

dist = [geopy.distance.vincenty((i,j), (k,l)).km for i, j, k, l in 
        list(zip(train['now_latitude'], train['now_longitude'], train['next_latitude'], train['next_longitude']))]
train['dist_line'] = dist

dist = [geopy.distance.vincenty((i,j), (k,l)).km for i, j, k, l in 
        list(zip(test['now_latitude'], test['now_longitude'], test['next_latitude'], test['next_longitude']))]
test['dist_line'] = dist

Y_train = list(train['next_arrive_time'])

train['day'] = pd.to_datetime(train['date']).dt.weekday_name
test['day'] = pd.to_datetime(test['date']).dt.weekday_name

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


In [25]:
up = (33.506286, 126.490312)
right = (33.493521, 126.895326)
center = (33.379724, 126.545315)

t1 = [geopy.distance.vincenty((i,j), up).km for i, j in list(zip(train['now_latitude'], train['now_longitude']))]
t3 = [geopy.distance.vincenty((i,j), right).km for i, j in list(zip(train['now_latitude'], train['now_longitude']))]
t4 = [geopy.distance.vincenty((i,j), center).km for i, j in list(zip(train['now_latitude'], train['now_longitude']))]
total = pd.DataFrame(list(zip(t1, t3, t4)), columns = ['up', 'right', 'center'])

train['dist_name'] = total.apply(lambda x : x.argmin(), axis = 1)

t1 = [geopy.distance.vincenty((i,j), up).km for i,j in list(zip(test['now_latitude'], test['now_longitude']))]
t3 = [geopy.distance.vincenty((i,j), right).km for i,j in list(zip(test['now_latitude'], test['now_longitude']))]
t4 = [geopy.distance.vincenty((i,j), center).km for i,j in list(zip(test['now_latitude'], test['now_longitude']))]

total = pd.DataFrame(list(zip(t1, t3, t4)), columns = ['up', 'right', 'center'])
test['dist_name'] = total.apply(lambda x : x.argmin(), axis = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


In [0]:
del train['now_latitude']
del train['now_longitude']
del train['next_latitude']
del train['next_longitude']
del train['id']
del train['date']
del train['route_id']
del train['vh_id']
del train['route_nm']

del test['route_id']
del test['vh_id']
del test['route_nm']
del test['now_latitude']
del test['now_longitude']
del test['next_latitude']
del test['next_longitude']
del train['next_arrive_time']
del test['date']
del test['id']


In [0]:
one_hot_train = pd.get_dummies(train)
one_hot_test = pd.get_dummies(test)
input_len = len(one_hot_train.columns)

X_train = one_hot_train.values.tolist()
X_test = one_hot_train.values.tolist()

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)

np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)
np.save('Y_train.npy', Y_train)

In [0]:
train_x = np.load()